In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime 
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import seaborn as sns

from lightgbm import LGBMRegressor
import lightgbm as lgb

In [30]:
train_features = pd.read_csv('../data/train_features.csv').drop(['Unnamed: 0'], axis=1)

In [31]:
train_features = train_features.drop(['uid','date','split', 'region', 'latitude', 'longitude'], axis=1)

In [32]:
l = []
for x in range(255):
    l.append(f'R_{x}')
    l.append(f'G_{x}')
    l.append(f'B_{x}')

In [33]:
train_features = train_features.drop(l, axis=1)

In [34]:
X = train_features.drop(['density'], axis=1)
Y = train_features['density']

In [7]:
features_name = np.array(X.columns)

In [35]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [10]:
gb = GradientBoostingRegressor()
gb_param_grid = {
    'n_estimators': [50,100,500,1000,1500,3000],
    'learning_rate': [0.05, 0.1, 0.5, 1],
    'max_features': [2,3,4,5,6,7],
    'min_samples_split': [2,3,5,10,15,20],
    'loss': ['absolute_error', 'squared_error', 'huber', 'quantile']}

gb_grid_search = GridSearchCV(estimator=gb,param_grid=gb_param_grid, n_jobs=-1, verbose=1)

In [11]:
gb_grid_search.fit(X_train, y_train)

gb_grid_search.best_params_

Fitting 5 folds for each of 3456 candidates, totalling 17280 fits


{'learning_rate': 0.1,
 'loss': 'huber',
 'max_features': 2,
 'min_samples_split': 15,
 'n_estimators': 3000}

In [12]:
y_pred = gb_grid_search.best_estimator_.predict(X_test)

In [13]:
metrics.root_mean_squared_error(y_test, y_pred)

1034435.4763280712

In [14]:
y_pred = gb_grid_search.best_estimator_.predict(X_train)

In [15]:
metrics.root_mean_squared_error(y_train, y_pred)

724852.3729795493

In [17]:
rf = RandomForestRegressor()

In [21]:
rf_params = {
    'n_estimators': [50,100,200,500],
    'max_depth': [3,4,5,6,7],
    'min_samples_split': [2,3,5,10,15,20],
    'criterion' :['squared_error', 'absolute_error', 'friedman_mse', 'poisson']
}

In [22]:
grid = GridSearchCV(estimator=rf,param_grid=rf_params, n_jobs=-1)

In [23]:
grid.fit(X_train, y_train)

grid.best_params_

NameError: name 'rid' is not defined

In [24]:

grid.best_params_

{'criterion': 'absolute_error',
 'max_depth': 7,
 'min_samples_split': 10,
 'n_estimators': 500}

In [43]:
y_pred = grid.best_estimator_.predict(X_train)

ValueError: X has 33 features, but RandomForestRegressor is expecting 35 features as input.

In [44]:
metrics.root_mean_squared_error(y_test, y_pred)

1289544.5400232507

In [37]:
rn = RandomForestRegressor(
    criterion='absolute_error',
    max_depth=7,
    min_samples_split=10,
    n_estimators=500
)

In [38]:
rn.fit(X_train, y_train)

RandomForestRegressor(criterion='absolute_error', max_depth=7,
                      min_samples_split=10, n_estimators=500)

In [42]:
y_pred = rn.predict(X_test)